![](https://www.pata.org/wp-content/uploads/2014/09/TripAdvisor_Logo-300x119.png)
# Predict TripAdvisor Rating
## В этом соревновании нам предстоит предсказать рейтинг ресторана в TripAdvisor
**По ходу задачи:**
* Прокачаем работу с pandas
* Научимся работать с Kaggle Notebooks
* Поймем как делать предобработку различных данных
* Научимся работать с пропущенными данными (Nan)
* Познакомимся с различными видами кодирования признаков
* Немного попробуем [Feature Engineering](https://ru.wikipedia.org/wiki/Конструирование_признаков) (генерировать новые признаки)
* И совсем немного затронем ML
* И многое другое...   


# import

In [69]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns 
%matplotlib inline
import re
import missingno as msno
sns.set()

from collections import Counter
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

# Загружаем специальный удобный инструмент для разделения датасета:
from sklearn.model_selection import train_test_split

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

pd.set_option("display.max_rows", 250)
pd.set_option("display.max_columns", 250)
# Any results you write to the current directory are saved as output.

In [70]:
# всегда фиксируйте RANDOM_SEED, чтобы ваши эксперименты были воспроизводимы!
RANDOM_SEED = 42

In [71]:
# зафиксируем версию пакетов, чтобы эксперименты были воспроизводимы:
!pip freeze > requirements.txt

# FUNCTIONS

In [72]:
# Create a list with restaurants which might be in chain
def show_chained_rest(data):
    chained_rest_list = list(data['Restaurant_id'].value_counts()[data['Restaurant_id'].value_counts() > 1].index)

In [73]:
#  Для определения границ выбросов
def func_IQR(data, column):
    perc_25 = data[column].quantile(0.25, interpolation="midpoint")
    perc_75 = data[column].quantile(0.75, interpolation="midpoint")
    IQR = perc_75 - perc_25
    print('Q1: {}'.format(perc_25), 'Q3: {}'.format(perc_75), 'IQR: {}'.format(IQR),
          'Emission limit: [{a},{b}]'.format(a=perc_25 - 1.5*IQR, b=perc_75 + 1.5*IQR), sep='\n')


In [74]:
# Processing the Cuisine Style column
# We transform the string with kitchens into a list of kitchens, for this we create a function
def str_to_list(row_list):
    if pd.isna(row_list): return ['unknown']
    row_list = row_list.replace("\'",'').strip('[]').split(', ')
    return row_list

# Add a column with the number of kitchens represented
def len_row(row): # counting the cuisines presented by the restaurant
    x = len(row)
    return x

    data['Cuisines'] = data['Cuisine Style'].apply(len_row)

# Adding a column for the availability of vegetarian food
def vegan(date):
    word = 'Veg'
    i=0
    for item in date:
        if word in item:
            i+=1
    return i

    data['Vegenarian_options'] = data['Cuisine Style'].apply(vegan)

# Add a column with the presence of Wine Bar or Pab
def wine(date):
    word = ['Wine Bar', 'Pub', 'Bar']
    i=0
    for item in date:
        if item in word:
            i+=1
    return i

    data['Wine_pub'] = data['Cuisine Style'].apply(wine)

# Add a column with Delicatessen
def delicatessen(date):
    word = 'Delicatessen'
    i=0
    for item in date:
        if item in word:
            i+=1
    return i

    data['delicatessen_pub'] = data['Cuisine Style'].apply(delicatessen)

# Add a column with the presence of Fast Food and Street Food
def fastfood(date):
    word = ['Fast Food', 'Street Food']
    i=0
    for item in date:
        if item in word:
            i+=1
    return i

    data['Fastfood'] = data['Cuisine Style'].apply(fastfood)

# Add a column with Sushi, Japanese
def sushi(date):
    word = ['Japanese', 'Sushi']
    i=0
    for item in date:
        if item in word:
            i+=1
    return i

    data['Sushi'] = data['Cuisine Style'].apply(sushi)

# Add a column with Grill, Barbecue
def grill(date):
    word = ['Grill', 'Barbecue']
    i=0
    for item in date:
        if item in word:
            i+=1
    return i

    data['Barbecue'] = data['Cuisine Style'].apply(grill)

# Add a column with the presence of Italian food
def italian(date):
    word = ['Pizza', 'Italian']
    i=0
    for item in date:
        if item in word:
            i+=1
    return i

    data['Italian_cuisine'] = data['Cuisine Style'].apply(italian)

# Add a column with the presence of European cuisine
def europe(date):
    word = 'Europe'
    i=0
    for item in date:
        if word in item:
            i+=1
    return i

    data['Europe_cuisine'] = data['Cuisine Style'].apply(europe)

# Add a column with the presence of south american cuisines
def south_american(date):
    word = ['Mexican', 'Venezuelan', 'South American', 'Brasilian']
    i=0
    for item in date:
        if item in word:
            i+=1
    return i

    data['South_american_cuisine'] = data['Cuisine Style'].apply(south_american)

# Adding a column for Asian food
def asian(date):
    word = ['Asian', 'Thai', 'Vietnamese']
    i=0
    for item in date:
        if item in word:
            i+=1
    return i

    data['Asian_cuisine'] = data['Cuisine Style'].apply(asian)
    
# Add a healthy kitchen column
def healthy(date):
    word = 'Healthy'
    i=0
    for item in date:
        if item in word:
            i+=1
    return i

    data['Healthy_cuisine'] = data['Cuisine Style'].apply(healthy)

# Adding a column with the presence of halal food
def halal(date):
    word = 'Halal'
    i=0
    for item in date:
        if item in word:
            i+=1
    return i

    data['Halal_cuisine'] = data['Cuisine Style'].apply(halal)

In [75]:
# Process the Reviews column
# Add a column with a positive feedback tone
# To determine the conditionally positive tone of the review
def str_to_list_rev(row_list):
    if pd.isna(row_list): return ['unknown']
    row_list = row_list.replace("\'",'').strip('[]').split(', ')
    return row_list

def good_rev(data):
    i = 0
    for item in data:
        checklist = {'good', 'nice', 'excellent', 'best', 'wonderful', 'unique', 'delicious','heavenly', 'amazing', 'brilliant', 'great place', 'better'}
        common_words = set(item.lower().split()) & checklist
        i += len(common_words)
    return i



In [76]:
# Replace the values of the categorical attribute Price Range with categories 1-3
def price_range(data):
    range_map = {'$':1, '$$ - $$$':2, '$$$$':3}
    data['Price Range'] = data['Price Range'].map(range_map)
    x_price = data['Price Range'].median()
    data['Price Range'].fillna(x_price, inplace=True)

In [77]:
# Add a sign of the difference in days between the submitted reviews
def days_bet_rev(data):
    pattern = re.compile('\d+\/\d+\/\d+')
    data['reviews_date'] = data['Reviews'].apply(lambda x: pattern.findall(x))
    
    def last_reviews (row): # Last_reviews column fill function
        if len(row) != 0:
            return row[0]
        else:
            return 0

    def first_reviews (row): # Function to fill the column first_reviews
        if len(row) != 0:
            return row[-1]
        else:
            return 0
# Add a sign of the difference in seconds between reviews
    data['first_reviews'] = data['reviews_date'].apply(first_reviews)
    data['last_reviews'] = data['reviews_date'].apply(last_reviews)
    data['last_reviews'] = pd.to_datetime(data['last_reviews']).dt.date
    data['first_reviews'] = pd.to_datetime(data['first_reviews']).dt.date
    data['days_bet_rev'] = data['last_reviews'] - data['first_reviews']
    data['days_bet_rev'] = np.abs(data['days_bet_rev'].dt.total_seconds()/86400)
# Delete temporary columns 
    data.drop(['last_reviews', 'first_reviews', 'reviews_date'], axis=1, inplace=True)

In [78]:
# Additional signs
# Index of purchasing power according to Numbeo rating (2018)
def signs_for_cities(data):
    purchasing_power_index = {
        'Amsterdam' : 19,
        'Athens' : 53,
        'Barcelona' : 30,
        'Berlin' : 8,
        'Bratislava' : 39,
        'Brussels' : 15,
        'Budapest' : 51,
        'Copenhagen' : 10,
        'Dublin' : 27,
        'Edinburgh' : 16,
        'Geneva' : 3,
        'Hamburg' : 7,
        'Helsinki' : 9,
        'Krakow' : 36,
        'Lisbon' : 48,
        'Ljubljana' : 32,
        'London' : 24,
        'Luxembourg' : 4,
        'Lyon' : 70,
        'Madrid' : 23,
        'Milan' : 40,
        'Munich' : 6,
        'Oporto' : 42,
        'Oslo' : 17,
        'Paris' : 22,
        'Prague' : 31,
        'Rome' : 37,
        'Stockholm' : 13,
        'Vienna' : 21,
        'Warsaw' : 29,
        'Zurich' : 2
    }

    data['purchasing_power_index'] = data['City'].map(purchasing_power_index)
    
# New trait - The number of inhabitants in the city, according to the Wiki
    population_dict = {'Paris': 2240621, 'Stockholm': 1981263, 'London': 8173900,\
                       'Berlin': 3326002, 'Munich': 1561094, 'Oporto': 258975,\
                       'Milan': 1331586, 'Bratislava': 413192, 'Vienna': 1765649,\
                       'Rome': 2870493, 'Barcelona': 1593075, 'Madrid': 3155360,\
                       'Dublin': 506211, 'Brussels': 144784, 'Zurich': 402275,\
                       'Warsaw': 1720398, 'Budapest': 1744665, 'Copenhagen': 1246611,\
                       'Amsterdam': 825080, 'Lyon': 496343, 'Hamburg':  1718187,\
                       'Lisbon': 547733, 'Prague': 1335084, 'Oslo': 673469,\
                       'Helsinki': 574579, 'Edinburgh': 468070,\
                       'Geneva': 196150, 'Ljubljana': 277554, 'Athens': 3168846,\
                       'Luxembourg': 648538, 'Krakow': 756183}
    data['population'] = data['City'].map(population_dict)
    

# Total restaurants in the city
    count_rest_dict = {'Paris': 15633, 'Stockholm': 2783, 'London': 17239,\
                       'Berlin': 5635, 'Munich': 2882, 'Oporto': 1861,\
                       'Milan': 6750, 'Bratislava': 1045, 'Vienna': 3803,\
                       'Rome': 10360, 'Barcelona': 8726, 'Madrid': 10348,\
                       'Dublin': 2107, 'Brussels': 2365, 'Zurich': 1676,\
                       'Warsaw': 2788, 'Budapest': 2786, 'Copenhagen': 1991,\
                       'Amsterdam': 3671, 'Lyon': 2574, 'Hamburg':  20457,\
                       'Lisbon': 4576, 'Prague': 4653, 'Oslo': 1190,\
                       'Helsinki': 1388, 'Edinburgh': 1773, 'Geneva': 1346,\
                       'Ljubljana': 542, 'Athens': 2362,\
                       'Luxembourg': 659, 'Krakow': 1535}
    data['all_rest'] = data['City'].map(count_rest_dict)

# Average salary in the country
    count_salary_dict = {'Paris': 3332, 'Stockholm': 2893, 'London': 2703,\
                         'Berlin': 4392, 'Munich': 4392, 'Oporto': 1288,\
                         'Milan': 2726, 'Bratislava': 1283, 'Vienna': 2940,
                         'Rome': 2726, 'Barcelona': 2133, 'Madrid': 2133,\
                         'Dublin': 2107, 'Brussels': 3930, 'Zurich': 6244,\
                         'Warsaw': 2788, 'Budapest': 2786, 'Copenhagen': 6192,\
                         'Amsterdam': 3238, 'Lyon': 3332, 'Hamburg':  4392,\
                         'Lisbon': 1288, 'Prague': 1354, 'Oslo': 5450,\
                         'Helsinki': 3908, 'Edinburgh': 2703, 'Geneva': 6244,\
                         'Ljubljana': 1914, 'Athens': 1203,'Luxembourg': 5854,\
                         'Krakow': 1253}
    data['average_salary'] = data['City'].map(count_salary_dict)
    
# Tourist attraction.
    # Place of the city in the ranking of the top 100 cities visited in 2018
    # Source - Euromonitor international
    # Since not all cities are in the top 100, we divide cities into groups:
    # 1 - place 1-25, 2 - place 26-50, 3 - place 51-75
    # 4 - place 76-100, 5 - place above 100

    tourist_attract = {'London' : 1, 'Paris' : 1, 'Madrid' : 2, 'Barcelona' : 2,\
        'Berlin' : 2, 'Milan' : 5, 'Rome' : 1, 'Prague' : 1, 'Lisbon' : 3, 'Vienna' : 2,\
        'Amsterdam' : 1, 'Brussels' : 3, 'Hamburg' : 5, 'Munich' : 3, 'Lyon' : 5,\
        'Stockholm' : 4, 'Budapest' : 3, 'Warsaw' : 3, 'Dublin' : 5, 'Copenhagen' : 3,\
        'Athens' : 2, 'Edinburgh' : 5, 'Zurich' : 5, 'Oporto' : 4, 'Geneva' : 5, \
        'Krakow' : 3, 'Oslo' : 5, 'Helsinki' : 5, 'Bratislava' : 5, 'Luxembourg' : 5,\
        'Ljubljana' : 5
    }

    data['tourist_attract'] = data['City'].map(tourist_attract)  

In [79]:
# function for standardization
def StandardScaler_column(data, d_col):
    scaler = StandardScaler()
    scaler.fit(data[[d_col]])
    return scaler.transform(data[[d_col]])

In [80]:
# Distribution of values ​​between Ranking and Raiting are similar.
# When restaurants with the highest Ranking get 4-4.5
# Create a feature that calculates the Rating value based on Ranking
# Compare the rank to the number of restaurants in the city.
# Normalize the data so that the rank does not depend on the number of restaurants in the city.
def ranking_normalize(data):
    mean_per_city = data.groupby('City')['Ranking'].mean()
    data['mean_ranking_per_city'] = data['City'].apply(lambda x: mean_per_city[x])
    max_per_city = data.groupby('City')['Ranking'].max()
    data['max_ranking_per_city'] = data['City'].apply(lambda x: max_per_city[x])
    data['standard_ranking'] = (data['Ranking'] - data['mean_ranking_per_city']) / data['max_ranking_per_city']
    data.drop(['mean_ranking_per_city', 'max_ranking_per_city'], axis = 1, inplace=True)
    

# DATA

In [81]:
DATA_DIR = '/kaggle/input/sf-dst-restaurant-rating/'
df_train = pd.read_csv(DATA_DIR+'/main_task.csv')
df_test = pd.read_csv(DATA_DIR+'kaggle_task.csv')
sample_submission = pd.read_csv(DATA_DIR+'/sample_submission.csv')

In [82]:
df_train.info()

In [83]:
df_train.head(5)

In [84]:
df_test.info()

In [85]:
df_test.head(5)

In [86]:
sample_submission.head(5)

In [87]:
sample_submission.info()

In [88]:
# IMPORTANT! For correct processing of features, we combine the train and test into one dataset
df_train['sample'] = 1 # mark where we have train
df_test['sample'] = 0 # mark where we have the test
df_test['Rating'] = 0 # in the test we do not have a Rating value, we have to predict it, so for now just fill in with zeros

data = df_test.append(df_train, sort=False).reset_index(drop=True) # combine

In [89]:
df = data.copy()
df.head()

Подробнее по признакам:
* City: Город 
* Cuisine Style: Кухня
* Ranking: Ранг ресторана относительно других ресторанов в этом городе
* Price Range: Цены в ресторане в 3 категориях
* Number of Reviews: Количество отзывов
* Reviews: 2 последних отзыва и даты этих отзывов
* URL_TA: страница ресторана на 'www.tripadvisor.com' 
* ID_TA: ID ресторана в TripAdvisor
* Rating: Рейтинг ресторана

In [90]:
df.sample(5)

In [91]:
df.Reviews[1]

As you can see, most of our signs require cleaning and preliminary processing.

# Cleaning and Prepping Data
Typically, the data contains a bunch of garbage that needs to be cleaned up in order to bring it into an acceptable format. Data cleansing is a necessary step in solving almost any real-world problem.  
![](https://analyticsindiamag.com/wp-content/uploads/2018/01/data-cleaning.png)

In [92]:
# Show number of missing percentages
display(df.isnull().sum()/(((df.index[-1])+1)/100))

# more clearly it is displayed on the graph
msno.bar(data, figsize=(18, 6),  fontsize=12);

In [93]:
data.Restaurant_id.value_counts()

We see a lot of duplicate ids, probably chain restaurants.

## 1. NAN processing
There can be various reasons for the presence of gaps, but gaps must be either filled in or excluded from the set entirely. But you need to be careful with gaps, ** even the lack of information can be an important sign! **
Therefore, before processing NAN, it is better to take out information about the presence of a pass as a separate sign.

In [94]:
# I'll take the Number of Reviews, Cuisine Style, Price Range column as an example
df['number_of_reviews_isNAN'] = pd.isna(df['Number of Reviews']).astype('uint8')
df['cuisine_style_isNAN'] = pd.isna(df['Cuisine Style']).astype('uint8')
df['price_range_isNAN'] = pd.isna(df['Price Range']).astype('uint8')
df['reviews_isNAN'] = pd.isna(df['Reviews']).astype('uint8')

In [95]:
df['number_of_reviews_isNAN']

In [96]:
# Next, fill in the gaps 0, you can try filling with the average or average for the city, etc.
df['Number of Reviews'].fillna((df['Number of Reviews'].mean()), inplace=True)# Show number of missing percentages
#df['Reviews'].fillna(('[], []]'), inplace=True)# Show number of missing percentages
df.isnull().sum()/(((df.index[-1])+1)/100)

### 2. Processing characteristics
First, let's see what signs we can have categorical.

In [97]:
df.nunique(dropna=False)

What signs can be considered categorical?
City, Price Range

There are many approaches to coding categorical features:
* Label Encoding
* One-Hot Encoding
* Target Encoding
* Hashing

The choice of coding depends on the characteristic and the chosen model.
Let's not dive deeply into this topic now, let's see a better example with One-Hot Encoding:
![](https://i.imgur.com/mtimFxh.png)

In [98]:
# For One-Hot Encoding, pandas has a ready-made function - get_dummies. The parameter dummy_na is especially pleasing.
df = pd.get_dummies(df, columns=[ 'City',], dummy_na=True)
df.info()

In [99]:
df.head(5)

In [100]:
df.sample(5)

#### Cuisine Style
Let's convert the values ​​of the Cuisine Style column to strings for further data processing.

In [101]:
df['Cuisine Style'] = df['Cuisine Style'].apply(str_to_list_rev)

#### Let's take the next attribute "Price Range".

In [102]:
df['Price Range'].value_counts()

### Processing the rest of the signs

In [103]:
# Create a dummy attribute from the "Price Range" categorical column
#df = pd.get_dummies(data=df, columns=['Price Range'], dummy_na=True)
# Create a 'price range' dictionary and replace the values in the current column
range_map = {'$':1, '$$ - $$$':2, '$$$$':3}
df['Price Range'] = df['Price Range'].map(range_map)

# fill in the gaps with median values
df['Price Range'].fillna((df['Price Range'].median()), inplace=True)

df.head()
    

In [104]:
df.info()

In [105]:
# CONVERT REVIEWS
# Make a pattern to extract dates from the Reviews string and create new features last_reviews and first_reviews
# We make a pattern to extract dates from the Reviews string and create new features
df['Reviews'].fillna('[Unknown]', inplace=True)
pattern = re.compile('\d+\/\d+\/\d+')
df['reviews_date'] = df['Reviews'].apply(lambda x: pattern.findall(x))


def last_reviews (row): # Function of the last_reviews
    if len(row) != 0:
        return row[0]
    else:
        return 0

    
def first_reviews (row): # Function of the first_reviews
    if len(row) != 0:
        return row[-1]
    else:
        return 0
    
    
df['first_reviews'] = df['reviews_date'].apply(first_reviews)
df['last_reviews'] = df['reviews_date'].apply(last_reviews)

# New feature - positive sentiment of reviews
df['reviews_temp'] = df['Reviews'].apply(str_to_list)

def good(date):
    i = 0
    for item in date:
        checklist = {'good', 'nice', 'excellent', 'best', 'wonderful', 'unique', 'delicious','heavenly', 'amazing', 'brilliant', 'great place', 'better'}
        common_words = set(item.lower().split()) & checklist
        i += len(common_words)
    return i

df['good_reviews'] = df['reviews_temp'].apply(good)

# New feature - with the difference in days between reviews
df['last_reviews'] = pd.to_datetime(df['last_reviews']).dt.date
df['first_reviews'] = pd.to_datetime(df['first_reviews']).dt.date
df['date_def'] = df['last_reviews'] - df['first_reviews']
df['date_def'] = df['date_def'].dt.total_seconds()/86400

In [106]:
df.info()

![](https://cs10.pikabu.ru/post_img/2018/09/06/11/1536261023140110012.jpg)

# EDA 
[Exploratory Data Analysis](https://ru.wikipedia.org/wiki/Разведочный_анализ_данных) - Data analysis
At this stage, we build graphs, look for patterns, anomalies, outliers or connections between signs.
In general, the purpose of this stage is to understand what this data can give us and how the signs can be interrelated.
Understanding the original features will allow us to generate new, stronger ones and, thereby, make our model better.
![](https://miro.medium.com/max/2598/1*RXdMb7Uk6mGqWqPguHULaQ.png)

### Let's see the distribution of the characteristic

In [107]:
plt.rcParams['figure.figsize'] = (10,7)
df_train['Ranking'].hist(bins=100)

We have a lot of restaurants that do not even hold up to 2500 seats in their city, but what about the cities?

In [108]:
df_train['City'].value_counts(ascending=True).plot(kind='barh')

And someone said that the French love to eat =) Let's see how the distribution in a big city will change:

In [109]:
df_train['Ranking'][df_train['City'] =='London'].hist(bins=100)

In [110]:
# посмотрим на топ 10 городов
for x in (df_train['City'].value_counts())[0:10].index:
    df_train['Ranking'][df_train['City'] == x].hist(bins=100)
plt.show()

It turns out that Ranking has a normal distribution, it is just that there are more restaurants in big cities, because of this we have an offset.

In [111]:
City_counts_dict = dict(df_train['City'].value_counts())
df_train['Restor_counts'] = df_train['City'].map(City_counts_dict)
df_train['Normalize_Ranking'] = df_train['Ranking']/df_train['Restor_counts']

Eliminating Ranking's dependence on the number of restaurants in the city

In [112]:
for x in (df_train['City'].value_counts())[0:10].index:
    df_train['Normalize_Ranking'][df_train['City'] == x].hist(bins=35)

### Let's see the distribution of the target variable (Rating)

In [113]:
df_train['Rating'].value_counts(ascending=True).plot(kind='barh')

### Let's see the distribution of the target variable Rating relative to the Ranking feature

In [114]:
df_train['Ranking'][df_train['Rating'] == 5].hist(bins=100)

In [115]:
df_train['Ranking'][df_train['Rating'] < 4].hist(bins=100)

### Feature correlation
In this graph, you can now see how the traits are related to each other and to the target variable.

In [116]:
plt.rcParams['figure.figsize'] = (15,10)
sns.heatmap(df.drop(['sample'], axis=1).corr(),)

# Data Preprocessing
Now, for convenience and code reproducibility, let's wrap all the processing in one big function.

In [117]:
# just in case, reload the data
DATA_DIR = '/kaggle/input/sf-dst-restaurant-rating/'
df_train = pd.read_csv(DATA_DIR+'/main_task.csv')
df_test = pd.read_csv(DATA_DIR+'/kaggle_task.csv')
df_train['sample'] = 1 # it's a train set
df_test['sample'] = 0 # it's a test set
df_test['Rating'] = 0 # in the test we do not have the Rating value, we must predict it, so for now we just fill it with zeros

df = df_test.append(df_train, sort=False).reset_index(drop=True) # объединяем
df.info()

In [118]:
def preproc_data(df_input):
    '''includes several functions to pre-process the predictor data.'''
    
    df_output = df_input.copy()
    
    # ################### 1. Preprocessing ############################################################## 
    # we remove the features that are not necessary for the model
    #df_output.drop(['Restaurant_id','ID_TA',], axis = 1, inplace=True)
    df_output['Restaurant_id'] = df_output['Restaurant_id'].apply(lambda x: float(x[3:]))
    df_output['ID_TA'] = df_output['ID_TA'].apply(lambda x: float(x[1:]))
    
    
    # ################### 2. NAN ############################################################## 
    df_output['Number of Reviews'].fillna(0, inplace=True)
    df_output['Number_of_Reviews_NaN'] = pd.isna(df_output['Number of Reviews']).astype('uint8')
    df_output['Cuisine_Style_NaN'] = pd.isna(df_output['Cuisine Style']).astype('uint8')
    df_output['Cuisine Style'] = df_output['Cuisine Style'].fillna("['Other']")
    df_output['Price_Range_NaN'] = pd.isna(df_output['Price Range']).astype('uint8')
    df_output['Reviews'] = df_output['Reviews'].fillna('[[], []]')
    df_output['Reviews_NaN'] = (df_output['Reviews'] == '[[], []]').astype('float64')
    
    # ################### 3. Encoding and Feature Engineering #########################################
    
    # Restaurant_id
    show_chained_rest(df_output)
    
    # City
    # Adding signs based on data about cities and countries
    signs_for_cities(df_output)
    
    
    
    # Cuisine Style
    df_output['Cuisine Style'] = df_output['Cuisine Style'].apply(str_to_list)
    len_row(df_output)
    vegan(df_output)
    wine(df_output)
    delicatessen(df_output)
    fastfood(df_output)
    sushi(df_output)
    grill(df_output)
    italian(df_output)
    europe(df_output)
    south_american(df_output)
    asian(df_output)
    healthy(df_output)
    halal(df_output)
    
    # Ranking
    ranking_normalize(df_output)
    
    # Price Range
    #price_range(df_output)
    df_output = pd.get_dummies(data=df_output, columns=['Price Range'], dummy_na=True)
    
    # Number of Reviews
    df_output['Number of Reviews'].fillna((df_output['Number of Reviews'].mean()), inplace=True)
    
    # Reviews
    days_bet_rev(df_output)
    good_rev(df_output)

    
    # URL_TA
    
    # ID_TA
    
    # New signs
    df_output['nor_population'] = df_output['Number of Reviews'] / df_output['population']*100000
    df_output['rank_per_restaurant'] = df_output['Ranking']/df_output.all_rest
    df_output['rank_per_city'] = df_output['Ranking']/df_output.population*1000
    df_output['rest_per_popul'] = df_output.all_rest/df_output.population*1000
    df_output['sol_per_popul_and_rest'] = (df_output.all_rest*df_output.average_salary)/df_output.population
    #df_output['num_per_rank_per_city'] = df_output.rank_per_restaurant/df_output['Number of Reviews']
   
    df_output = pd.get_dummies(data=df_output, columns=['City',], dummy_na=True)
    
    
    # ################### 4. Clean #################################################### 
    # remove signs that have not yet been processed,
    # model on features with dtypes "object" will not be trained, just select them and delete
    object_columns = [s for s in df_output.columns if df_output[s].dtypes == 'object']
    df_output.drop(object_columns, axis = 1, inplace=True)
    
    
    # Feature standardization
    object_columns = [s for s in df_output.columns if df_output[s].dtypes == 'object']

    # Standardize all columns except target and Sample
    for i in list(df_output.columns):
        if i not in ['Rating','sample'] and i not in object_columns:
            df_output[i] = StandardScaler_column(df_output, i)
            if len(df_output[df_output[i].isna()]) < len(df_output):
                df_output[i] = df_output[i].fillna(0)
                
                

    return df_output

df_preproc = preproc_data(df)
df_preproc.sample(10)

>On the good, it would be possible to translate this large function into a class and break it up into subfunctions (according to OOP).

#### Launch and check what happened

In [119]:
df_preproc = preproc_data(df)
df_preproc.sample(10)

In [120]:
df_preproc.info()

In [121]:
# Now let's select the test part
train_data = df_preproc.query('sample == 1').drop(['sample'], axis=1)
test_data = df_preproc.query('sample == 0').drop(['sample'], axis=1)

y = train_data.Rating.values            # our target
X = train_data.drop(['Rating'], axis=1)

** Before sending our data for training, let's divide the data into one more test and train for validation.
This will help us test how well our model is performing before submitting the submission to kaggle. **

In [122]:
# Let's use the special function train_test_split to split test data
# allocate 20% of the data for validation (parameter test_size)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_SEED)

In [123]:
# testing
test_data.shape, train_data.shape, X.shape, X_train.shape, X_test.shape

# Model 
Сам ML

In [124]:
# Import the required libraries:
from sklearn.ensemble import RandomForestRegressor # tool for creating and training a model
from sklearn import metrics # tools for assessing model accuracy

In [125]:
# Create a model (DO NOT TOUCH SETTINGS)
model = RandomForestRegressor(n_estimators=100, verbose=1, n_jobs=-1, random_state=RANDOM_SEED)

In [126]:
# Train the model on the test dataset
model.fit(X_train, y_train)

# We use a trained model to predict restaurant ratings in a test sample.
# Predicted values ​​are written to the y_pred variable
y_pred = model.predict(X_test)



In [127]:
# It can be observed that the difference in that real ratings are always multiples of 0.5
# Write a function to round the predicted ratings accordingly
def round_rating_pred(rating_pred):
    if rating_pred <= 0.25:
        return 0.0
    if rating_pred <= 0.75:
        return 0.5
    if rating_pred <= 1.25:
        return 1.0
    if rating_pred <= 1.75:
        return 1.5
    if rating_pred <= 2.25:
        return 2.0
    if rating_pred <= 2.75:
        return 2.5
    if rating_pred <= 3.25:
        return 3.0
    if rating_pred <= 3.75:
        return 3.5
    if rating_pred <= 4.25:
        return 4.0
    if rating_pred <= 4.75:
        return 4.5
    return 5.0


# Round it
for i in range(len(y_pred)):
    y_pred[i] = round_rating_pred(y_pred[i])

In [128]:
# Compare the predicted values ​​(y_pred) with the real ones (y_test), and see how much they differ on average
# The metric is called the Mean Absolute Error (MAE) and shows the average deviation of the predicted values ​​from the actual ones.
print('MAE:', metrics.mean_absolute_error(y_test, y_pred))

In [129]:
# in RandomForestRegressor it is possible to display the most important features for the model
plt.rcParams['figure.figsize'] = (10,10)
feat_importances = pd.Series(model.feature_importances_, index=X.columns)
feat_importances.nlargest(15).plot(kind='barh')

# Submission
Готовим Submission на Kaggle

In [130]:
test_data.sample(10)

In [131]:
test_data = test_data.drop(['Rating'], axis=1)

In [132]:
sample_submission

In [133]:
predict_submission = model.predict(test_data)

In [134]:
predict_submission = list(map(round_rating_pred, predict_submission))

In [135]:
sample_submission['Rating'] = predict_submission
sample_submission.to_csv('submission.csv', index=False)
sample_submission.head(10)